In [17]:
from random import seed
from random import randrange
from math import sqrt

import numpy as np 
import pandas as pd 
from csv import reader
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.utils import shuffle



In [18]:
# Pre Processing
class NormStandart():
    def __init__(self, dataset, dsformat):
        self.dataset = dataset
        self.format = dsformat

    def normalize(self, dataset):
        max = dataset.max()
        min = dataset.min()
        for i in range(len(dataset.transpose()[0])):
            for j in range(4):
                dataset[i][j] = (dataset[i][j] - min)/(max-min)
        return dataset


    def mean(self, dataset):
        col_quant = len(dataset[0])
        dataset = dataset.transpose()
        means = np.zeros(col_quant)
        for i in range(col_quant):
            means[i] = np.sum(dataset[i])/float(len(dataset[i]))
        return means

    # calculate column standard deviations
    def stdev(self, dataset, means):
        col_quant = len(dataset[0])
        stdevs = np.zeros(col_quant)
        for i in range(col_quant):
            variance = [pow(row[i]-means[i], 2) for row in dataset]
            stdevs[i] = sum(variance)
        stdevs = [sqrt(x/(float(len(dataset)-1))) for x in stdevs]
        return np.array(stdevs)

    def standardize_dataset(self, dataset, means, stdevs):
        for row in dataset:
            for i in range(len(row)):
                row[i] = (row[i] - means[i]) / stdevs[i]

    def normalize_and_standartize(self):

        self.dataset = self.normalize(self.dataset)
        means = self.mean(self.dataset)
        stdevs = self.stdev(self.dataset, means)

        for row in self.dataset:
            for i in range(len(row)):
                row[i] = (row[i] - means[i]) / stdevs[i]
        if self.format == "numpy":
            return self.dataset
        elif self.format == "pandas":
            return pd.DataFrame(data=self.dataset)

ds = pd.read_csv("winequality-white.csv", sep=";", dtype=np.float32, engine='c', float_precision='round-trip')
print(ds.head(10))

ns = NormStandart(ds.to_numpy(), "pandas")
dataset= ns.normalize_and_standartize()
print(dataset.head(10))


# Y = dataset.loc[:, 4]
# X = dataset.iloc[:, :-1]

# # # insert 1 in every row for intercept b
# X.insert(loc=len(X.columns), column=4, value=1)

# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# print("Train Data")
# print(X_train.head(5))
# print(y_train.head(5))

# print("Test Data")
# print(X_test.head(5))
# print(y_test.head(5))


   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.0              0.27         0.36       20.700001      0.045   
1            6.3              0.30         0.34        1.600000      0.049   
2            8.1              0.28         0.40        6.900000      0.050   
3            7.2              0.23         0.32        8.500000      0.058   
4            7.2              0.23         0.32        8.500000      0.058   
5            8.1              0.28         0.40        6.900000      0.050   
6            6.2              0.32         0.16        7.000000      0.045   
7            7.0              0.27         0.36       20.700001      0.045   
8            6.3              0.30         0.34        1.600000      0.049   
9            8.1              0.22         0.43        1.500000      0.044   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 45.0                 170.0   1.0010  3.00       0

In [42]:
# Split a dataset into k folds

def cross_validation_split(dataset, k_folds):
    rows = dataset.shape[0]
    fold_size = round(rows/k_folds)
    dataset_split = []
    for i in range(0, rows, fold_size):
        if rows - (i+fold_size) <= 0:
            dataset_split.append(dataset.loc[i:])
            break
        dataset_split.append(dataset.loc[i:i+fold_size-1])
    return dataset_split

print(dataset.shape)
print(cross_validation_split(dataset, 5))

(4898, 12)
[            0         1         2         3         4         5         6  \
0    0.172079 -0.081761  0.213259  2.821061 -0.035351  0.569873  0.744489   
1   -0.657434  0.215874  0.047996 -0.944669  0.147732 -1.252891 -0.149669   
2    1.475601  0.017450  0.543783  0.100272  0.193503 -0.312109 -0.973236   
3    0.409083 -0.478609 -0.117266  0.415726  0.559670  0.687471  1.120977   
4    0.409083 -0.478609 -0.117266  0.415726  0.559670  0.687471  1.120977   
..        ...       ...       ...       ...       ...       ...       ...   
975  0.883091  0.215874 -0.695684 -0.412341  0.376586 -0.782500 -0.643809   
976 -0.301929 -1.470726 -0.282528 -0.294045  0.559670  0.687471 -0.173200   
977 -0.420430 -1.371514 -0.034635  0.238283  0.193503  1.040264  1.191568   
978  1.001593  0.017450 -0.034635  0.060840 -0.401518 -0.194512  0.391532   
979 -1.012940 -0.776244  3.105347 -0.944669  4.770588 -1.488086 -1.985047   

            7         8         9        10        11  
0    2.

In [107]:
# Make a prediction with coefficients
def predict(row, weights):
    try:
        row_value = weights[0]
        for i in range(row.shape[0]-1):
            row_value += weights[i + 1] * row[i]
        return row_value
    except:
        print(row)

# Estimate linear regression coefficients using stochastic gradient descent
def weights_sgd(train, l_rate, n_epoch):
#     weights = [0.0 for i in range(train.shape[0])]
    weights = np.zeros(train.shape[0])
    for epoch in range(n_epoch-1):
        for index, row in train.iterrows():
            row_value = predict(row, weights)
            error = row_value - row.iloc[-1]
            weights[0] = weights[0] - l_rate * error
            for i in range(row.shape[0]-1):
                weights[i + 1] = weights[i + 1] - l_rate * error * row[i]
        print("Epoch {}/{}, Error: {}".format(epoch, n_epoch, error))
        print("Weights: {}".format(weights))
    return weights

# Linear Regression Algorithm With Stochastic Gradient Descent
def linear_regression_sgd(train, test, l_rate, n_epoch):
    predictions = list()
    weights = weights_sgd(train, l_rate, n_epoch)
    for row in test:
        row_value = predict(row, weights)
        predictions.append(row_value)
    return(predictions)

In [110]:
# Calculate root mean squared error
def rmse_metric(actual, predicted):
    sum_error = 0.0
    for i in range(len(actual)):
        prediction_error = predicted[i] - actual[i]
        sum_error += (prediction_error ** 2)
    mean_error = sum_error / float(len(actual))
    return sqrt(mean_error)

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, k_folds, *args):
    folds = cross_validation_split(dataset, k_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = pd.concat(train_set)
        test_set = list()
        for index, row in fold.iterrows():
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row.iloc[-1] for index, row in fold.iterrows()]
        rmse = rmse_metric(actual, predicted)
        scores.append(rmse)
    return scores

In [111]:
n_folds = 5
l_rate = 0.02
n_epoch = 50
scores = evaluate_algorithm(dataset, linear_regression_sgd, n_folds, l_rate, n_epoch)
print('Scores: %s' % scores)
print('Mean RMSE: %.3f' % (sum(scores)/float(len(scores))))

Epoch 0/50, Error: 0.46480907355917567
Weights: [-0.11151038  0.05490544 -0.19786463 ...  0.          0.
  0.        ]
Epoch 1/50, Error: 0.472905325607037
Weights: [-0.11343425  0.06992858 -0.19718911 ...  0.          0.
  0.        ]
Epoch 2/50, Error: 0.4737899487707642
Weights: [-0.11364446  0.07157006 -0.1971153  ...  0.          0.
  0.        ]
Epoch 3/50, Error: 0.473886605608663
Weights: [-0.11366742  0.07174941 -0.19710724 ...  0.          0.
  0.        ]
Epoch 4/50, Error: 0.4738971666528409
Weights: [-0.11366993  0.07176901 -0.19710636 ...  0.          0.
  0.        ]
Epoch 5/50, Error: 0.47389832058728043
Weights: [-0.11367021  0.07177115 -0.19710626 ...  0.          0.
  0.        ]
Epoch 6/50, Error: 0.47389844666995495
Weights: [-0.11367024  0.07177138 -0.19710625 ...  0.          0.
  0.        ]
Epoch 7/50, Error: 0.4738984604461627
Weights: [-0.11367024  0.07177141 -0.19710625 ...  0.          0.
  0.        ]
Epoch 8/50, Error: 0.47389846195139695
Weights: [-0.113

TypeError: unsupported operand type(s) for -: 'NoneType' and 'float'

In [3]:
!pip install mxnet

     |████████████████████████████████| 68.7 MB 5.3 MB/s eta 0:00:01


In [49]:
# Linear Regression With Stochastic Gradient Descent for Wine Quality
from random import seed
from random import randrange
from csv import reader
from math import sqrt
import numpy as np 
import pandas as pd 



# Load a CSV file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file, delimiter=';')
        csv_headings = next(csv_reader)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

# Convert string column to float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

# Find the min and max values for each column
def dataset_minmax(dataset):
    minmax = list()
    for i in range(len(dataset[0])):
        col_values = [row[i] for row in dataset]
        value_min = min(col_values)
        value_max = max(col_values)
        minmax.append([value_min, value_max])
    return minmax

# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
    for row in dataset:
        for i in range(len(row)):
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

# # Split a dataset into k folds
# def cross_validation_split(dataset, n_folds):
#     dataset_split = list()
#     dataset_copy = list(dataset)
#     fold_size = int(len(dataset) / n_folds)
#     for i in range(n_folds):
#         fold = list()
#         while len(fold) < fold_size:
#             index = randrange(len(dataset_copy))
#             fold.append(dataset_copy.pop(index))
#         dataset_split.append(fold)
#     return np.array([np.array(xi) for xi in dataset_split])

def cross_validation_split(dataset, k_folds):
    rows = dataset.shape[0]
    fold_size = round(rows/k_folds)
    dataset_split = []
    for i in range(0, rows, fold_size):
#         if rows - (i+fold_size) <= 0:
#             ds = dataset.loc[i:]
#             for i in range(-(rows - (i+fold_size))):
#                 zeros = pd.DataFrame(np.zeros(dataset.shape[1]))
#                 ds = pd.concat([ds, zeros], ignore_index=True)
#             dataset_split.append(ds)
#             break
        if dataset.loc[i:i+fold_size-1].shape[0] != fold_size:
            break
        dataset_split.append(dataset.loc[i:i+fold_size-1].to_numpy())
    return dataset_split

# Calculate root mean squared error
def rmse_metric(actual, predicted):
    sum_error = 0.0
    for i in range(len(actual)):
        prediction_error = predicted[i] - actual[i]
        sum_error += (prediction_error ** 2)
    mean_error = sum_error / float(len(actual))
    return sqrt(mean_error)

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
#         train_set.remove(fold)
        try:
            train_set = [ x for x in train_set if not (x==fold).all()]
        except:
            print(train_set[1].shape, fold.shape)
        train_set = np.concatenate(train_set, axis=0)
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        test = np.array([np.array(xi) for xi in test_set])
        train = np.array([np.array(xi) for xi in train_set])
        predicted = algorithm(train, test, *args)
        actual = [row[-1] for row in fold]
        rmse = rmse_metric(actual, predicted)
        scores.append(rmse)
    return scores

# Make a prediction with coefficients
def predict(row, coefficients):
    yhat = coefficients[0]
    for i in range(len(row)-1):
        yhat += coefficients[i + 1] * row[i]
    return yhat

# Estimate linear regression coefficients using stochastic gradient descent
def coefficients_sgd(train, l_rate, n_epoch):
    coef = [0.0 for i in range(len(train[0]))]
    for epoch in range(n_epoch):
        for row in train:
            yhat = predict(row, coef)
            error = yhat - row[-1]
            coef[0] = coef[0] - l_rate * error
            for i in range(len(row)-1):
                coef[i + 1] = coef[i + 1] - l_rate * error * row[i]
    return coef

# Linear Regression Algorithm With Stochastic Gradient Descent
def linear_regression_sgd(train, test, l_rate, n_epoch):
#     test = np.array([np.array(xi) for xi in test])
#     train = np.array([np.array(xi) for xi in train])
    predictions = np.zeros(test.shape[0])
    coef = coefficients_sgd(train, l_rate, n_epoch)
    i = 0
    for row in test:
        yhat = predict(row, coef)
        predictions[i] = yhat
        i+=1
    return(predictions)

# # Linear Regression on wine quality dataset
# seed(1)
# # load and prepare data
# filename = 'winequality-white.csv'
# dataset = load_csv(filename)
# for i in range(len(dataset[0])):
#     str_column_to_float(dataset, i)
# # normalize
# minmax = dataset_minmax(dataset)
# normalize_dataset(dataset, minmax)
ds = pd.read_csv("winequality-white.csv", sep=";", dtype=np.float32, engine='c', float_precision='round-trip')

ns = NormStandart(ds.to_numpy(), "pandas")
dataset= ns.normalize_and_standartize()
# evaluate algorithm
n_folds = 5
l_rate = 0.01
n_epoch = 50
scores = evaluate_algorithm(dataset, linear_regression_sgd, n_folds, l_rate, n_epoch)
print('Scores: %s' % scores)
print('Mean RMSE: %.3f' % (sum(scores)/float(len(scores))))

Scores: [1.023087523535337, 0.9992647309157645, 0.906128454717904, 0.8586404195909617]
Mean RMSE: 0.947
